In [6]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow as tf
sns.set()

In [7]:
import kagglehub
import os

# Download latest version of the dataset
path = kagglehub.dataset_download("murtozalikhon/brain-tumor-multimodal-image-ct-and-mri")

print("Path to dataset files:", path)

# Inspect the dataset structure
print("\nExploring the dataset structure...\n")
for folder in os.listdir(path):
    print(f"Folder: {folder}")
    subfolder_path = os.path.join(path, folder)
    if os.path.isdir(subfolder_path):
        for subfolder in os.listdir(subfolder_path):
            print(f"  Subfolder: {subfolder}")
            image_files = os.listdir(os.path.join(subfolder_path, subfolder))
            print(f"    Number of images: {len(image_files)}")


c:\Users\supra\.conda\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1

Exploring the dataset structure...

Folder: Dataset
  Subfolder: Brain Tumor CT scan Images
    Number of images: 3
  Subfolder: Brain Tumor MRI images
    Number of images: 3


In [18]:
path = r' C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1\Dataset'
ct_path = os.path.join(path, "Brain Tumor CT scan Images")
mri_path = os.path.join(path, "Brain Tumor MRI images")


In [19]:
import os
import pandas as pd
import glob

# Define dataset paths
mri_healthy_path = r"C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1\Dataset\Brain Tumor MRI images\Healthy"
mri_tumor_path = r"C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1\Dataset\Brain Tumor MRI images\Tumor"
ct_healthy_path = r"C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1\Dataset\Brain Tumor CT scan Images\Healthy"
ct_tumor_path = r"C:\Users\supra\.cache\kagglehub\datasets\murtozalikhon\brain-tumor-multimodal-image-ct-and-mri\versions\1\Dataset\Brain Tumor CT scan Images\Tumor"

# Initialize data list
data = []

# Add MRI Healthy
print("Adding MRI Healthy images...")
for img_path in glob.glob(os.path.join(mri_healthy_path, "*.jpg")):
    data.append((img_path, 0, "MRI"))
print(f"Added {len(data)} MRI Healthy images.")

# Add MRI Tumor
print("Adding MRI Tumor images...")
for img_path in glob.glob(os.path.join(mri_tumor_path, "*.jpg")):
    data.append((img_path, 1, "MRI"))
print(f"Total images after MRI Tumor: {len(data)}")

# Add CT Healthy
print("Adding CT Healthy images...")
for img_path in glob.glob(os.path.join(ct_healthy_path, "*.jpg")):
    data.append((img_path, 0, "CT"))
print(f"Total images after CT Healthy: {len(data)}")

# Add CT Tumor
print("Adding CT Tumor images...")
for img_path in glob.glob(os.path.join(ct_tumor_path, "*.jpg")):
    data.append((img_path, 1, "CT"))
print(f"Total images after CT Tumor: {len(data)}")

# Create DataFrame
df = pd.DataFrame(data, columns=["path", "label", "modality"])
print("\nSample DataFrame:")
print(df.head())

# Check label distribution
print("\nLabel Distribution:")
print(df['label'].value_counts())

# Check modality distribution
print("\nModality Distribution:")
print(df['modality'].value_counts())


Adding MRI Healthy images...
Added 1997 MRI Healthy images.
Adding MRI Tumor images...
Total images after MRI Tumor: 4981
Adding CT Healthy images...
Total images after CT Healthy: 6697
Adding CT Tumor images...
Total images after CT Tumor: 8854

Sample DataFrame:
                                                path  label modality
0  C:\Users\supra\.cache\kagglehub\datasets\murto...      0      MRI
1  C:\Users\supra\.cache\kagglehub\datasets\murto...      0      MRI
2  C:\Users\supra\.cache\kagglehub\datasets\murto...      0      MRI
3  C:\Users\supra\.cache\kagglehub\datasets\murto...      0      MRI
4  C:\Users\supra\.cache\kagglehub\datasets\murto...      0      MRI

Label Distribution:
label
1    5141
0    3713
Name: count, dtype: int64

Modality Distribution:
modality
MRI    4981
CT     3873
Name: count, dtype: int64


In [20]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def preprocess_data(df, img_size):
    images = []
    labels = []
    for _, row in df.iterrows():
        # Load the image
        img = cv2.imread(row['path'], cv2.IMREAD_COLOR)
        if img is None:  # Handle failed image loading
            print(f"Warning: Failed to load image at {row['path']}")
            continue
        # Resize the image
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(row['label'])
    if len(images) == 0:  # Handle case where no images are loaded
        raise ValueError("No images were loaded. Please check your dataset paths and formats.")
    images = np.array(images) / 255.0  # Normalize
    labels = to_categorical(labels, num_classes=2)  # One-hot encode
    return images, labels

# Preprocess data for MRI and CT
img_size = 224  # MobileNet input size

# Process MRI data
try:
    X_mri, y_mri = preprocess_data(df[df['modality'] == 'MRI'], img_size)
except ValueError as e:
    print(f"Error processing MRI data: {e}")
    X_mri, y_mri = np.array([]), np.array([])

# Process CT data
try:
    X_ct, y_ct = preprocess_data(df[df['modality'] == 'CT'], img_size)
except ValueError as e:
    print(f"Error processing CT data: {e}")
    X_ct, y_ct = np.array([]), np.array([])

# Check if datasets are not empty before splitting
if X_mri.size > 0 and y_mri.size > 0:
    X_train_mri, X_val_mri, y_train_mri, y_val_mri = train_test_split(
        X_mri, y_mri, test_size=0.2, random_state=42
    )
    print(f"MRI - Training samples: {len(X_train_mri)}, Validation samples: {len(X_val_mri)}")
else:
    print("MRI data is empty. Skipping training/validation split for MRI.")

if X_ct.size > 0 and y_ct.size > 0:
    X_train_ct, X_val_ct, y_train_ct, y_val_ct = train_test_split(
        X_ct, y_ct, test_size=0.2, random_state=42
    )
    print(f"CT - Training samples: {len(X_train_ct)}, Validation samples: {len(X_val_ct)}")
else:
    print("CT data is empty. Skipping training/validation split for CT.")


MRI - Training samples: 3984, Validation samples: 997
CT - Training samples: 3098, Validation samples: 775


In [21]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

def build_mobilenet_model(input_shape):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze base model layers for transfer learning
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (img_size, img_size, 3)

# Build models for MRI and CT
mri_model = build_mobilenet_model(input_shape)
ct_model = build_mobilenet_model(input_shape)


17225924/17225924 [==============================] - 3s 0us/step


In [25]:
# Train MRI model
print("Training MRI Model...")
history_mri = mri_model.fit(
    X_train_mri, y_train_mri,
    validation_data=(X_val_mri, y_val_mri),
    epochs=10,
    batch_size=32
)

# Train CT model
print("Training CT Model...")
history_ct = ct_model.fit(
    X_train_ct, y_train_ct,
    validation_data=(X_val_ct, y_val_ct),
    epochs=10,
    batch_size=32
)


Training MRI Model...
Epoch 1/10
125/125 [==============================] - 5s 40ms/step - loss: 0.0916 - accuracy: 0.9671 - val_loss: 0.0677 - val_accuracy: 0.9749
Epoch 2/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0980 - accuracy: 0.9639 - val_loss: 0.0668 - val_accuracy: 0.9779
Epoch 3/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0843 - accuracy: 0.9701 - val_loss: 0.0720 - val_accuracy: 0.9719
Epoch 4/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0884 - accuracy: 0.9671 - val_loss: 0.0706 - val_accuracy: 0.9749
Epoch 5/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0761 - accuracy: 0.9741 - val_loss: 0.0645 - val_accuracy: 0.9789
Epoch 6/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0775 - accuracy: 0.9716 - val_loss: 0.0635 - val_accuracy: 0.9779
Epoch 7/10
125/125 [==============================] - 4s 30ms/step - loss: 0.0788 - accuracy: 0.9684 - val_loss: 0.0656 

In [26]:
mri_model.save('mri_model_mobilenet.h5')
ct_model.save('ct_model_mobilenet.h5')


In [28]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

def predict_tumor_single(mri_model_path, ct_model_path, img_path, img_size=224):
    
    # Load the models
    mri_model = load_model(mri_model_path)
    ct_model = load_model(ct_model_path)
    
    # Load and preprocess the image
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"Could not read the image at {img_path}")
    
    img = cv2.resize(img, (img_size, img_size)) / 255.0  # Resize and normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    
    # Predict using both models
    mri_pred = mri_model.predict(img)[0][1]  # Tumor probability from MRI model
    ct_pred = ct_model.predict(img)[0][1]  # Tumor probability from CT model
    
    # Combine predictions (average probabilities)
    combined_pred = (mri_pred + ct_pred) / 2
    
    # Return True if tumor is detected, False otherwise
    return combined_pred > 0.5


In [31]:
# Paths to the trained models
mri_model_path = "mri_model_mobilenet.h5"
ct_model_path = "ct_model_mobilenet.h5"

# Path to the input image
input_image = r"C:\machine learning]\brain tumor\ct_healthy (20).png"

# Predict
try:
    tumor_detected = predict_tumor_single(mri_model_path, ct_model_path, input_image)
    print(f"Image: {input_image} -> Tumor Detected: {tumor_detected}")
except ValueError as e:
    print(e)


1/1 [==============================] - 0s 283ms/step
Image: C:\machine learning]\brain tumor\ct_healthy (20).png -> Tumor Detected: False
